In [1]:
import pandas as pd

In [2]:
questions = pd.read_csv("data/questions.csv")
instructions = pd.read_csv("data/instructions.csv")

In [3]:
len(questions), len(instructions)

(8312, 8312)

In [4]:
questions.iloc[7598]["content"]

'Divide the letter "N" into halves.\n\n[STRDGM] The letter "N". [STPDGM]'

In [74]:
questions.head()

,subject,exercise_type,difficulty,grade_level,modifier,content
0,Computer Science,"Practical Exercise, Activity",D,2,M,Create a document file called 'My hobby' to us...
1,Computer Science,Algorithm Writing Exercise,D,2,Exercise on making a toy horse,Write an algorithm for making a toy horse. \nW...
2,Computer Science,Short Answer Exercise,D,2,With Material,What does ‘algorithm' mean? Write or say the a...
3,Computer Science,Short Answer Exercise,D,5,With Material,Imagine that the x coordinate of an apple is a...
4,Computer Science,Programming Exercise,D,5,"With Material, Exercise on sprite creation and...","Add a sprite (for example, an apple) to your s..."


In [5]:
for i in range(len(instructions)):
    q_exercise_types = questions.iloc[i]["exercise_type"].split(",")
    # print(q_exercise_types)
    [t.lower() in instructions.iloc[i]["Input"].lower() for t in q_exercise_types]
    if all(t.lower() in instructions.iloc[i]["Input"].lower() for t in q_exercise_types):
        # print(True, i)
        pass
    else:
        print(False, i)
    # break

I have a dataframe with 5 columns: "subject"	"exercise_type"	"difficulty"	"grade_level"	"modifier"	"content".
Now I want to randomly sample 2 rows and calculate the number of matching columns ("content" is exlcluded), for instance, row1["subject"] == row2["subject"] --> 1 matching
Create a dataframe to store sample results, each result contains "num_matched_column", "row1.index", "row2.index"

In [33]:
import pandas as pd
import numpy as np
import random

# Define function to sample and calculate matches
def sample_and_match(df):

    # With 0.2 probability take the input output from the same sample to have score of 4
    if random.random() < 0.2:
        sampled_rows = df.sample(1)
        idx1 = idx2 = sampled_rows.index[0]
        num_matched_columns = 4
        result = {
            'num_matched_column': num_matched_columns,
            'row1.index': idx1,
            'row2.index': idx2
        }
    else:
        # Sample 2 rows
        sampled_rows = df.sample(2)
        
        # Extract the indices of the sampled rows
        idx1, idx2 = sampled_rows.index
        
        # Exclude "content" and "difficulty" column from the comparison
        cols_to_compare = df.columns.difference(['content', 'difficulty'])
        
        # Count the number of matching columns (excluding "content" and "difficulty")
        num_matched_columns = np.sum(sampled_rows.iloc[0][cols_to_compare] == sampled_rows.iloc[1][cols_to_compare])
        
        # Create a result dictionary
        result = {
            'num_matched_column': num_matched_columns,
            'row1.index': idx1,
            'row2.index': idx2
        }
    
    return result

# Example usage
result_df = sample_and_match(questions)
print(result_df)

{'num_matched_column': 4, 'row1.index': 6359, 'row2.index': 6359}


In [34]:
questions.iloc[result_df["row1.index"]], questions.iloc[result_df["row2.index"]]

(subject                                                Mathematics
 exercise_type                                  Arithmetic Exercise
 difficulty                                                       D
 grade_level                                                      2
 modifier         With Illustration, Exercise on counting, Multi...
 content          Start at 0. Draw 4 jumps of 5 on a number line...
 Name: 6359, dtype: object,
 subject                                                Mathematics
 exercise_type                                  Arithmetic Exercise
 difficulty                                                       D
 grade_level                                                      2
 modifier         With Illustration, Exercise on counting, Multi...
 content          Start at 0. Draw 4 jumps of 5 on a number line...
 Name: 6359, dtype: object)

In [36]:
# Create 100 samples and store results in a DataFrame
results = []
for _ in range(20000):
    result = sample_and_match(questions)
    results.append(result)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Display the resulting DataFrame
# print(results_df)

# Display the distribution of different values of num_matched_column
distribution = results_df['num_matched_column'].value_counts().sort_index()

# Display the distribution
print(distribution)

num_matched_column
0    10434
1     4442
2      978
3      113
4     4033
Name: count, dtype: int64


In [42]:
# Sample 50 values from each category of num_matched_column
def sample_from_each_category(df, column, sample_size=50):
    # Group by the specified column (num_matched_column)
    grouped = df.groupby(column)
    
    # Sample up to 'sample_size' rows from each group
    sampled_df = grouped.apply(lambda x: x.sample(min(len(x), sample_size), replace=False)).reset_index(drop=True)
    
    return sampled_df

# Apply the function to sample 50 rows from each category of num_matched_column
sampled_results_df = sample_from_each_category(results_df, 'num_matched_column', sample_size=100)

# Display the new DataFrame with the sampled results
print(sampled_results_df['num_matched_column'].value_counts().sort_index())

num_matched_column
0    100
1    100
2    100
3    100
4    100
Name: count, dtype: int64


/tmp/ipykernel_2576612/3838881311.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = grouped.apply(lambda x: x.sample(min(len(x), sample_size), replace=False)).reset_index(drop=True)


In [6]:
# sampled_results_df.to_csv("data/coherence_eval.csv", index=False)
sampled_results_df = pd.read_csv("data/coherence_eval.csv")

In [7]:
# Sample "Input" based on row1.index and "Output" based on row2.index
sampled_results_df['Input'] = sampled_results_df['row1.index'].apply(lambda idx: instructions.loc[idx, 'Input'])
sampled_results_df['Output'] = sampled_results_df['row2.index'].apply(lambda idx: instructions.loc[idx, 'Output'])

In [9]:
shuffle_data = sampled_results_df.sample(frac=1, ignore_index=True)

In [ ]:
import openai

# Set up your OpenAI API key
openai.api_key = "sk-proj-ltmkMm6qZ8oQCsusN5IOT3BlbkFJmsPopivPYwLtY7jlx5Pl"

# Example to generate text using GPT-4
def generate_text(sys_prompt, user_prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",  # Use "gpt-4-turbo" for a faster, cheaper alternative
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=150,  # Limit the number of tokens in the output
            temperature=0.7,  # Adjust the creativity level of the model
            n=1,  # Number of completions to generate
            stop=None  # Stop generation based on special characters or tokens
        )
        return response['choices'][0]['message']['content'].strip()
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
prompt = "Write a short story about a robot learning to play the piano."
generated_text = generate_text(prompt)
print(generated_text)


In [ ]:
system_prompt = """You are an exercise generation evaluator. 
Given an query-answer pair, where the query is a demand and the answer is the generated exercise,
you will score the coherence of the excercise from 4 perspectives:
- Subject: score 1 if the subject is correct else 0.
- Exercise type: score 1 if the exercise type is correct else 0.
- Grade level: score 1 if the grade level is close to the given according to 1-12 Amecian grade system else 0.
- Modification: score 1 if the specific details matches the requirement else 0.
Don't be too strict to each criterion, you can give them score as long as it is full filled in a cetain level. Your final score is the sum of all these four scores. Provide your answer in the following format:

{"Subject score": x, "Exercise type score": x, "Grade level": x, "Modification score": x, "Total score": x}"""

def parse_response(response):
    pass

In [ ]:
predict_scores = []
for index, row in sampled_results_df.iterrows():
    input, output, score = row['Input'], row['Output'], row['num_matched_column']
    user_prompt = f"Query: \"{input}\"\n\nAnswer: \"{output}\""
    response = generate_text(system_prompt, user_prompt)
    result = parse_response(response)
    predict_scores.append(result["Total score"])
    break

In [ ]:
final_results_df = pd.DataFrame({"predict_score": predict_scores, "true_score": sampled_results_df['num_matched_column'].values})

### Parse and build questions dataframe with param columns

In [37]:
import re
import pandas as pd

def parse_params(text):
    text = text.strip()
    # Find content '(...)' or S or T or D or G or M or number (5 params)
    params= re.findall(r"\(.+?\)|\d+|S|T|D|G|M", text)
    params = [p.replace("(","").replace(")", "") for p in params]
    # assert len(params) == 5, "Parse result is not complete"
    assert len(params) == 5, text
    return params

with open("data/questions.txt", "r") as f:
    text = f.read()

text = text
title_pattern = r"\([a-zA-Z].+\).+ D .+\n"
# Find all titles (questions)
titles = re.findall(title_pattern, text)
# Find all contents (answers)
contents = re.split(title_pattern, text)
contents = [content.strip() for content in contents]
contents = contents[1:] if contents[0] == "" else contents

In [ ]:



text = text
title_pattern = r"\([a-zA-Z].+\).+ D .+\n"
# Find all titles (questions)
titles = re.findall(title_pattern, text)
# Find all contents (answers)
contents = re.split(title_pattern, text)
contents = [content.strip() for content in contents]
contents = contents[1:] if contents[0] == "" else contents






In [38]:
subject, exercise_type, difficulty, grade_level, modifier = ([] for _ in range(5))

for title in titles:
    params = parse_params(title)
    subject.append(params[0])
    exercise_type.append(params[1])
    difficulty.append(params[2])
    grade_level.append(params[3])
    modifier.append(params[4])

struct_data = pd.DataFrame({"subject": subject, "exercise_type": exercise_type, "difficulty": difficulty,
                             "grade_level": grade_level, "modifier": modifier, "content": contents})

struct_data.to_csv("data/questions.csv",index=False)